In [ ]:
import sys, os
sys.path.append(os.path.join(os.path.dirname("__file__"), '..'))
sys.path.append(os.path.join(os.path.dirname("__file__"), '..', '..'))
sys.path.append(os.path.join(os.path.dirname("__file__"), '..', '..', '..'))

import io
import argparse
import pickle
import random
import pdb
import time
import numpy as np

import torch
from torch.utils.data import DataLoader
from torch import optim
from torch.nn import MSELoss, L1Loss
import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')
import torch.nn.functional as F
from deepsnap.batch import Batch as deepsnap_Batch
from le_pde.argparser import arg_parse
from le_pde.models import load_model, get_model
from le_pde.utils import EXP_PATH
from le_pde.pytorch_net.util import filter_filename, ddeepcopy as deepcopy, init_args
from le_pde.datasets.load_dataset import load_data
from le_pde.utils import to_tuple_shape, get_pos_dims_dict, get_device, get_precision_floor
from le_pde.PDE_Control.legacy.phi.fluidformat import *
from le_pde.PDE_Control.legacy.phi.flow import FluidSimulation, DomainBoundary
from le_pde.PDE_Control.legacy.phi.math.nd import *
from le_pde.pytorch_net.util import get_decay_list
from le_pde.pytorch_net.util import to_np_array

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from IPython.core.display import display, HTML

display(HTML("<style>div.output_scroll { height: unset; }</style>"))

args = arg_parse()
args.exp_id = "smoke"
args.date_time = "2022-3-6"
args.dataset="movinggas"
args.n_train= "-1"
args.time_interval= 1
args.save_interval= 10
args.algo= "contrast"
args.reg_type= None
args.reg_coef= 0
args.is_reg_anneal= True
args.no_latent_evo= False
args.encoder_type= "cnn-s"
args.evolution_type= "mlp-3-elu-2"
args.decoder_type= "cnn-tr"
args.encoder_n_linear_layers= "0"
args.n_conv_blocks= 4
args.n_latent_levs= 2
args.n_conv_layers_latent= 3
args.channel_mode= "exp-16"
args.is_latent_flatten= False
args.evo_groups= 1
args.recons_coef= 1
args.consistency_coef= 1
args.contrastive_rel_coef= 0
args.hinge= 0
args.density_coef= 0.001
args.latent_noise_amp= "1e-5"
args.normalization_type= "gn"
args.latent_size= 16
args.kernel_size= 4
args.stride= 2
args.padding= 1
args.padding_mode= "zeros"
args.act_name= "elu"
args.multi_step= "1"
args.latent_multi_step= "1"
args.use_grads= False
args.use_posargs= False
args.is_y_diff= False
args.loss_type= "mse"
args.loss_type_consistency= "mse"
args.batch_size= 2
args.val_batch_size= 8
args.epochs= 52
args.opt= "adam"
args.weight_decay= 0
args.disc_coef= 0
args.seed= 0
args.gpuid= 3
args.id= 0
args.is_train=False
args.is_test_only=True


In [ ]:
def rollout(model, preds, info, data, is_recons):
    """Go to original representation."""
    info["input"] = data.node_feature
    precision_floor = get_precision_floor(model.loss_type)
    if model.loss_type is not None and precision_floor is not None:
        preds_core = {}
        if is_recons and "recons" in info:
            recons_core = {}
        for loss_type_key in model.loss_type.split("^"):
            key = loss_type_key.split(":")[0]
            if "mselog" in loss_type_key or "huberlog" in loss_type_key or "l1log" in loss_type_key:
                if len(preds) > 0 and len(preds[key]) > 0:
                    preds_core[key] = torch.exp(preds[key]) - precision_floor
                if is_recons and "recons" in info:
                    recons_core[key] = torch.exp(info["recons"][key]) - precision_floor
            else:
                if len(preds) > 0:
                    preds_core[key] = preds[key]
                if is_recons and "recons" in info:
                    recons_core[key] = info["recons"][key]
        preds = preds_core
        if is_recons and "recons" in info:
            info["recons"] = recons_core
    return preds, info

class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else: return super().find_class(module, name)

In [ ]:
# Settings:

exp_id = "smoke"
date_time = "2022-3-28"
# include is a list of strings that the filename must include 
include = [
    ".p",
    "movinggas_train_-1_algo_contrast_ebm_False_ebmt_cd_enc_cnn-s_evo_cnn_act_elu_hid_128_lo_mse_recef_1.0_conef_1.0_nconv_4_nlat_1_clat_3_lf_True_reg_None_id_0_Hash_TgzjEJou_turing4.p",
]

# Load dataset:
dataset_name = "movinggas"
print(date_time)
(dataset_train_val, dataset_test), (train_loader, val_loader, test_loader) = load_data(args)


init_indices = [0]  # starting t
n_rollout_steps = 70  # number of rollout steps
interval = 1         # Interval to visualize
is_test = True      # Whether to use the test dataset

# Run analysis:
dirname = "results/" + "{}_{}/".format(exp_id, date_time)
filenames = filter_filename(dirname, include=include)
args_list = list(arg_parse().__dict__.keys())
dataset = deepcopy(dataset_test) if is_test else deepcopy(dataset_train_val)
isplot=2
n_plots_row=6
print(filenames)


In [ ]:
if len(filenames)==0:
    raise Exception("No file in the current path. Check if your current path, that is dirname, is correct!")

original_shape = dataset[0].original_shape
is_gpu=True
if is_gpu:
     for filename in filenames:
        data_record = pickle.load(open(dirname + filename, "rb"))
        args = init_args(data_record["args"])
        device = "cuda:0"
        #device = get_device(args)
        print(data_record["best_model_dict"]["type"])
        try:
            #pdb.set_trace()
            model = load_model(data_record["best_model_dict"], device, input_shape=original_shape)
        except Exception as e:
            raise
            print("Cannot load '{}'".format(filename))
            continue
else:
    device = torch.device("cpu")
    for filename in filenames:
        f = open(dirname + filename, "rb")
        data_record = CPU_Unpickler(f).load()
        try:
            model = load_model(data_record["best_model_dict"], device, input_shape=original_shape)
        except Exception as e:
            raise
            print("Cannot load '{}'".format(filename))
            continue

model.model_dict["input_size"]
model.eval()

In [ ]:
from PDE_Control.legacy.phi.math.nd import *
from PDE_Control.legacy.phi.solver.sparse import SparseCGPressureSolver
from PDE_Control.legacy.phi.fluidformat import *

def initialize_velocity():
    vx = np.random.uniform(low=2.99, high=2.99)
    vy = np.random.uniform(low=0., high=0.)
    velocity_array = np.empty([129, 129, 2], np.float32)
    velocity_array[...,0] = vx
    velocity_array[...,1] = vy
    init_op_velocity = StaggeredGrid(velocity_array.reshape((1,)+velocity_array.shape))
    optimizable_velocity = init_op_velocity.staggered
    return init_op_velocity, optimizable_velocity

def apply_mask(sim, optimizable_velocity):
    ###Set Initial Condition for Velocity###
    control_mask = sim.ones("staggered")
    control_mask.staggered[:, 4:117, 18:-18, :] = 0
    divergent_velocity = optimizable_velocity * control_mask.staggered
    divergent_velocity = StaggeredGrid(divergent_velocity)
    return divergent_velocity

def plot_velocity_with_mask(divergent_velocity):
    fig, ax = plt.subplots(figsize=(8,4),ncols=2)
    ###Heatmap of initial velocity in x-dirction###
    mappable0 = ax[0].imshow(divergent_velocity.staggered[0,:,:,0], cmap='viridis',
                             #extent=[0,sensordata.shape[0],0,sensordata.shape[1]],
                             aspect='auto',
                             origin='lower')
    ###Heatmap of initial velocity in y-dirction###
    mappable1 = ax[1].imshow(divergent_velocity.staggered[0,:,:,1], cmap='viridis',
                             #extent=[0,sensordata.shape[0],0,sensordata.shape[1]],
                             interpolation="bicubic",
                             aspect='auto',
                             origin='lower')
    fig.colorbar(mappable0, ax=ax[0])
    fig.colorbar(mappable1, ax=ax[1])
    fig.tight_layout()
    plt.show()
    
def plot_velocity_boundary_effect(velocity):
    hor_velocity_array = np.empty([129, 129, 1], np.float32)
    hor_velocity_array[...,0] = velocity.staggered[0,:,:,0]

    ver_velocity_array = np.empty([129, 129, 1], np.float32)
    ver_velocity_array[...,0] = velocity.staggered[0,:,:,1]


    fig, ax = plt.subplots(figsize=(8,4),ncols=2)
    ###Heatmap of velocity meating equation in x-dirction###
    mappable0 = ax[0].imshow(hor_velocity_array, cmap='viridis',
                             #extent=[0,sensordata.shape[0],0,sensordata.shape[1]],
                             aspect='auto',
                             origin='lower')
    ###Heatmap of velocity meating equation in y-dirction###
    mappable1 = ax[1].imshow(ver_velocity_array, cmap='viridis',
                             #extent=[0,sensordata.shape[0],0,sensordata.shape[1]],
                             interpolation="bicubic",
                             aspect='auto',
                             origin='lower')
    fig.colorbar(mappable0, ax=ax[0])
    fig.colorbar(mappable1, ax=ax[1])
    fig.tight_layout()
    plt.show()
    
def plot_initial_velocity(sim):
    fig_ob, ax_ob = plt.subplots(figsize=(4,4))
    ###Heatmap of initial velocity in x-dirction###
    mappable_ob0 = ax_ob.imshow(sim._active_mask[0,:,:,0], cmap='viridis',
                             #extent=[0,sensordata.shape[0],0,sensordata.shape[1]],
                             aspect='auto',
                             origin='lower')
    fig_ob.colorbar(mappable_ob0, ax=ax_ob)
    fig_ob.tight_layout()
    plt.show()

def plot_init_op_velocity(init_op_velocity):
    fig, ax = plt.subplots(figsize=(8,4),ncols=2)
    ###Heatmap of initial velocity in x-dirction###
    mappable0 = ax[0].imshow(init_op_velocity.staggered[0,:,:,0], cmap='viridis',
                             #extent=[0,sensordata.shape[0],0,sensordata.shape[1]],
                             aspect='auto',
                             origin='lower')
    ###Heatmap of initial velocity in y-dirction###
    mappable1 = ax[1].imshow(init_op_velocity.staggered[0,:,:,1], cmap='viridis',
                             #extent=[0,sensordata.shape[0],0,sensordata.shape[1]],
                             interpolation="bicubic",
                             aspect='auto',
                             origin='lower')
    fig.colorbar(mappable0, ax=ax[0])
    fig.colorbar(mappable1, ax=ax[1])
    fig.tight_layout()
    plt.show()
    
def plot_vector_field(velocity):
    fig = plt.figure()
    x,y = np.meshgrid(np.linspace(0,128,129),np.linspace(0, 128, 129))
    xvel = np.zeros([129]*2)
    yvel = np.zeros([129]*2)
    xvel[1::8,1::8] = velocity.staggered[0,1::8,1::8,0]
    yvel[1::8,1::8] = velocity.staggered[0,1::8,1::8,1]

    plt.quiver(x,y,xvel,yvel,scale=2.5, scale_units='inches')
    plt.show()

def get_beta_list(start_value, end_value, steps, mode="const"):
    if mode == "const":
        beta_list = [0.051 for _ in range(steps)]
    elif mode == "linear":
        beta_list = np.linspace(start_value, end_value, steps)
    elif mode == "exp":
        beta_list = np.logspace(np.log(start_value), np.log(end_value), steps)
    elif mode == "cos":
        from plasma.pytorch_net.util import get_cosine_decay
        beta_list = get_cosine_decay(start_value, end_value, steps)
    else:
        raise
    return beta_list

def diffble_map_to_grid(x0, x1, beta, hor=[], ver=[]):

    grid = np.zeros([128,128], dtype=np.float32)

    small = x0
    large = x1

    sigmoid = torch.nn.Sigmoid()
    lineval_list = []
    for i in range(128):
        if i==0:
            if i <= small:
                thor_tensor = sigmoid((i-small)/beta)
            elif i >= large:
                thor_tensor = sigmoid((large-i)/beta)
            else:
                denominator = 1/torch.abs(i-small) + 1/torch.abs(i-large)
                thor_tensor = sigmoid(2/(beta*denominator))
            hor_tensor = thor_tensor
        else:    
            if i <= small:
                temp_tensor = sigmoid((i-small)/beta)
                hor_tensor = torch.cat((hor_tensor, temp_tensor))
            elif i >= large:
                temp_tensor = sigmoid((large-i)/beta)
                hor_tensor = torch.cat((hor_tensor,temp_tensor))
            else:
                denominator = 1/torch.abs(i-small) + 1/torch.abs(i-large)
                hor_tensor = torch.cat((hor_tensor, sigmoid(2/(beta*denominator))))

    if len(hor) > 0:
        thor_tensor = hor_tensor.reshape(1,128)
        p2d = (0, 0, int(hor[0]), int(128 - hor[0] - 1))
        line_ingrid = F.pad(thor_tensor, p2d, "constant", 0.)
    elif len(ver) > 0:
        thor_tensor = hor_tensor.reshape(128, 1)
        p2d = (int(ver[0]), int(128 - ver[0] - 1), 0, 0)
        line_ingrid = F.pad(thor_tensor, p2d, "constant", 0.)
    else:
        raise Exception("Either of hor or ver for diffble_map_to_grid must be non-empty.")

    return line_ingrid

def continuous_bound(lx1, rx1, rx2, beta, lver=20, rver=110, bhor=4, uhor=115):
    #left boundary
    lx0 = torch.tensor([4. - 2.]).to(device)
    l_inlet = torch.tensor([20.]).to(device)
    lx2 = torch.tensor([115. + 2.]).to(device)

    left_bottom = diffble_map_to_grid(lx0, lx0+lx1, beta, hor=[], ver=[lver])
    left_upper = diffble_map_to_grid(lx0+lx1+l_inlet, lx2, beta, hor=[], ver=[lver])

    #right_boundary
    rx0 = torch.tensor([4. - 2.]).to(device)
    r_outlet1 = torch.tensor([20.]).to(device)

    r_outlet2 = torch.tensor([20.]).to(device)
    rx3 = torch.tensor([115. + 2.]).to(device)

    rx01 = torch.max(torch.cat((rx0+rx1, rx0 + 5.), -1), -1).values.reshape((1,))
    right_bottom = diffble_map_to_grid(rx0, rx01, beta, ver=[rver])
    rxmin = torch.max(torch.cat((rx2, torch.tensor([5.]).to(device)), -1), -1).values.reshape((1,))
    right_middle = diffble_map_to_grid(rx01+r_outlet1, rx01+r_outlet1+rxmin, beta, ver=[rver])
    upper_bottom = torch.min(torch.cat((rx01+r_outlet1+rxmin+r_outlet2, rx3 - 5.), -1), -1).values.reshape((1,))
    right_upper = diffble_map_to_grid(upper_bottom, rx3, beta, ver=[rver])

    #ceiling
    ceil0 = torch.tensor([20. - 2.]).to(device)
    ceil1 = torch.tensor([110. + 2.]).to(device)

    ceiling = diffble_map_to_grid(ceil0, ceil1, beta, hor=[uhor])

    #bottom
    bott0 = torch.tensor([20. - 2.]).to(device)
    bott1 = torch.tensor([110. + 2.]).to(device)

    bottom = diffble_map_to_grid(bott0, bott1, beta, hor=[bhor])

    left_bound = torch.max(torch.cat((left_bottom.reshape(left_bottom.shape+(1,)), left_upper.reshape(left_upper.shape+(1,))), -1), -1).values
    right_bound = torch.max(
        torch.cat(
            (right_bottom.reshape(right_bottom.shape+(1,)),
             right_middle.reshape(right_middle.shape+(1,)),
             right_upper.reshape(right_upper.shape+(1,))
            ), -1
        ), -1
                          ).values

    lr_bound = torch.max(torch.cat((left_bound.reshape(left_bound.shape+(1,)), right_bound.reshape(right_bound.shape+(1,))), -1), -1).values

    cb_bound = torch.max(torch.cat((ceiling.reshape(ceiling.shape+(1,)), bottom.reshape(bottom.shape+(1,))), -1), -1).values

    full_bound = torch.max(torch.cat((lr_bound.reshape(lr_bound.shape+(1,)), cb_bound.reshape(cb_bound.shape+(1,))), -1), -1).values

    return full_bound

def make_thick_bound(lx1, rx1, rx2, lver=20, rver=110, bhor=4, uhor=115, beta=0.001):
    full_bound = continuous_bound(lx1, rx1, rx2, beta)
    full_bound_l1 = continuous_bound(lx1, rx1, rx2, beta, lver=20-1, rver=110-1, bhor=4+1, uhor=115-1)
    full_bound_l2 = continuous_bound(lx1, rx1, rx2, beta, lver=20-2, rver=110-2, bhor=4+2, uhor=115-2)
    full_bound_r1 = continuous_bound(lx1, rx1, rx2, beta, lver=20+1, rver=110+1, bhor=4+3, uhor=115+1)

    thick_bound = torch.max(torch.cat((full_bound.reshape((128,128,1)), full_bound_l1.reshape((128,128,1)), full_bound_l2.reshape((128,128,1)), full_bound_r1.reshape((128,128,1))), -1), -1).values

    return 1. - thick_bound

def outlet_masks(lx1, rx1, rx2):
    lx0 = torch.tensor([4. - 2.]).to(device)
    l_inlet = torch.tensor([20.]).to(device)
    lx2 = torch.tensor([115. + 2.]).to(device)

    #right_boundary
    rx0 = torch.tensor([4. - 2.]).to(device)
    r_outlet1 = torch.tensor([20.]).to(device)

    r_outlet2 = torch.tensor([20.]).to(device)
    rx3 = torch.tensor([115. + 2.]).to(device)

    rx01 = torch.max(torch.cat((rx0+rx1, rx0 + 5.), -1), -1).values.reshape((1,))
    rxmin = torch.max(torch.cat((rx2, torch.tensor([5.]).to(device)), -1), -1).values.reshape((1,))
    upper_bottom = torch.min(torch.cat((rx01+r_outlet1+rxmin+r_outlet2, rx3 - 5.), -1), -1).values.reshape((1,))

    mask_out1 = torch.zeros((128, 128), dtype=torch.float32).to(device)
    mask_out2 = torch.zeros((128, 128), dtype=torch.float32).to(device)

    mask_out1[int(torch.round(rx01)[0])+4-2:int(torch.round(rx01)[0])+4+20+2, 108:112] = 1.
    mask_out2[int(torch.round(rx01+r_outlet1+rxmin)[0]):int(torch.round(rx01+r_outlet1+rxmin+r_outlet2)[0])+3, 108:112] = 1.

    return mask_out1, mask_out2

def initialize_gas():
    array = np.zeros([128, 128, 1], np.float32)
    m = margin = 10
    start_x = np.random.randint(24, 40)
    start_y = np.random.randint(8+m, 100)
    array[start_y:start_y+11, start_x:start_x+11, :] = 1
    print(array.shape)
    return array

def put_gas(start_x, inlet_coord):
    array = np.zeros([128, 128, 1], np.float32)
    m = margin = 10
    #start_x = np.random.randint(24, 40)
    start_y = int(inlet_coord)
    #start_y = inlet_coord + np.random.randint(-5, 5)
    #start_y = np.random.randint(8+m, 100)
    array[start_y:start_y+11, start_x:start_x+11, :] = 1
    print(array.shape)
    return array


def velocity_update(np_bound):
    sim = FluidSimulation([128]*2, DomainBoundary([(True, True), (True, True)]), force_use_masks=True)
    sim._active_mask[0,:,:,0] = np_bound
    sim._fluid_mask[0,:,:,0] = np_bound
    sim._velocity_mask = sim._boundary.create_velocity_mask(sim._fluid_mask, sim._dimensions, sim._mac)

    init_op_velocity, optimizable_velocity = initialize_velocity()
    divergent_velocity = apply_mask(sim, optimizable_velocity)
    #plot_velocity_with_mask(divergent_velocity)
    velocity = sim.divergence_free(divergent_velocity, solver=SparseCGPressureSolver(), accuracy=1e-5)
    velocity = sim.with_boundary_conditions(velocity)
    vel_x = velocity.staggered[0,:-1,:-1,0]
    vel_y = velocity.staggered[0,:-1,:-1,1]
    return vel_x, vel_y

def outlet_loss(preds, masks, preds_length):
    gas_scenes = preds["n0"][:,:,0]
    inners1 = []
    inners2 = []
    for i in range(preds_length):
        inners1.append(torch.inner(gas_scenes[:,i], masks[0].reshape(-1)))
        inners2.append(torch.inner(gas_scenes[:,i], masks[1].reshape(-1)))
    gas_out1 = torch.stack(inners1).sum()
    gas_out2 = torch.stack(inners2).sum()

    pred_fractions = torch.cat((gas_out1.reshape((1,)), gas_out2.reshape((1,))))
    return torch.div(pred_fractions, pred_fractions.sum())


init_step=89
orgdata = deepcopy(dataset[init_step]).to(device)
def make_data(data, thick_bound):
    ta = thick_bound.reshape([16384, 1, 1])
    init_gas = orgdata.node_feature["n0"][:,:,1].reshape([16384, 1, 1])
    tb = orgdata.node_feature["n0"][:,:,2].reshape([16384, 1, 1])
    tc = orgdata.node_feature["n0"][:,:,3].reshape([16384, 1, 1])
    data.node_feature["n0"]=torch.cat((ta, init_gas, tb, tc), -1)
    return data


Parameters for Optimization

In [ ]:
#parameters
#device='cuda:2'
is_recons=False
is_test_pred=False
is_binary_thick_bound=False
target_length=69
args.latent_multi_step=str(target_length)
iterations = 100
time_draw=10
args.entropy_coef = 1e+3
preds_length=20
drawstep=60
loss = MSELoss()
fdata = deepcopy(dataset[init_step]).to(device)
stdata = deepcopy(dataset[init_step]).to(device)
#beta=0.051
beta_list = get_decay_list(0.1, 0.051, iterations, "linear")
print(dataset)

In [ ]:
def randomize_gas(init_gas, rand=None):
    re_gas = init_gas.reshape([128,128]).cpu().detach().numpy()
    indices = np.where(re_gas != 0.)
    
    if rand != None:
        ran_y = rand[0] # np.random.randint(-5, 5)
        ran_x = rand[1] # np.random.randint(-5, 5)
        new_indices = (indices[0] + ran_y, indices[1] + ran_x)
    else:    
        new_indices = (indices[0], indices[1])
        
    rand_init_gas = np.zeros([128, 128], dtype = np.float32)
    rand_init_gas[new_indices] = re_gas[indices]
    
    return torch.tensor(rand_init_gas).to(device)


# optimization

In [ ]:
num_samples = 20
test_time = 20

lx1_range = [79., 80.]
rx1_range = [44., 45., 46., 47., 48., 49., 50.]
rx2_range = [16.]
y_direc = [-1, 0, 1]
x_direc = [0, 1]

from itertools import product
sample_space = list(product(lx1_range, rx1_range, rx2_range, y_direc, x_direc))
size_samspace = len(sample_space)

sample_indices = np.random.choice(size_samspace, size=50, replace=False)

In [ ]:
original_shape = dict(to_tuple_shape(fdata.original_shape))
pos_dims = get_pos_dims_dict(original_shape)
grid_keys = fdata.grid_keys

pre_defined_fraction = torch.tensor([0.3, 0.7], dtype=torch.float32).to(device)

all_pred_fractions = []
all_test_fractions = []
all_test_params = []
all_losses = []
all_times = []

params_dict = {}
for i in range(num_samples):
    print(time.time(), "sampled_params: ", sample_space[sample_indices[i]])

    pred_fraction1 = []
    pred_fraction2 = []
    target_fractions1 = []
    target_fractions2 = []

    loss_list = []
    #entropy_list = []

    best_preds = None
    best_loss = np.infty
    best_fractions = []
    best_params = ()
    times = []

    lx1 = torch.tensor([sample_space[sample_indices[i]][0]]).to(device)
    lx1.requires_grad=True
    rx1 = torch.tensor([sample_space[sample_indices[i]][1]]).to(device) #  <--- best_configuration
    rx1.requires_grad=True
    rx2 = torch.tensor([16.]).to(device) #  <--- best_configuration
    rx2.requires_grad=True
    optimizer = torch.optim.Adam([lx1, rx1, rx2], lr=3.7e-1) # <--- best_configuration

    params_test = [(torch.clone(lx1).detach(), torch.clone(rx1).detach(), torch.clone(rx2).detach())]

    rand = [sample_space[sample_indices[i]][3], sample_space[sample_indices[i]][4]]
    org_gas = orgdata.node_feature["n0"][:,:,1]
    plt.imshow(to_np_array(org_gas.reshape([128,128])), origin="lower")
    plt.show()
    for it in range(iterations):
        print("iteration_"+str(it))
        if ((it+1)%test_time == 0):
            params_test.append((torch.clone(lx1).detach(), torch.clone(rx1).detach(), torch.clone(rx2).detach()))
            print("params_test: ", params_test)

        thick_bound = make_thick_bound(lx1, rx1, rx2, beta=beta_list[it])
        if is_binary_thick_bound:
            binary_thick_bound = torch.where(thick_bound>0.5, 1., 0.)
        else:
            binary_thick_bound = thick_bound
        np_bound = to_np_array(binary_thick_bound)

        if it%time_draw==0:
            fig_t0, axt0 = plt.subplots()
            mappablet0 = axt0.imshow(np_bound, origin="lower")
            fig_t0.colorbar(mappablet0, ax=axt0)
            plt.show()
        # pdb.set_trace()
        vel_x, vel_y = velocity_update(np_bound)
        masks = outlet_masks(lx1, rx1, rx2)

        if is_test_pred:
            data=orgdata
        else:
            fdata = deepcopy(dataset[init_step]).to(device)
            fdata = make_data(fdata, binary_thick_bound)
            ta = fdata.node_feature["n0"][:,:,0].reshape([16384, 1, 1])
            #init_gas = fdata.node_feature["n0"][:,:,1].reshape([16384, 1, 1])
            rand_gas = randomize_gas(org_gas, rand)
            print("rand_gas")
            plt.imshow(to_np_array(rand_gas), origin="lower")
            plt.show()
            init_gas = rand_gas.reshape([16384,1,1])
            tb = torch.tensor(vel_y).reshape([16384, 1, 1]).to(device)
            tc = torch.tensor(vel_x).reshape([16384, 1, 1]).to(device)
            fdata.node_feature["n0"]=torch.cat((ta, init_gas, tb, tc), -1)

            stdata = deepcopy(dataset[init_step]).to(device)
            stdata=make_data(stdata, binary_thick_bound)
            ta = stdata.node_feature["n0"][:,:,0].reshape([16384, 1, 1])
            init_gas = stdata.node_feature["n0"][:,:,1].reshape([16384, 1, 1])
            tb = torch.tensor(vel_y).reshape([16384, 1, 1]).to(device)
            tc = torch.tensor(vel_x).reshape([16384, 1, 1]).to(device)
            stdata.node_feature["n0"]=ta

    #     if it%time_draw==0:
    #         size = torch.Size([1,4])
    #         print(*size)
    #         a = to_np_array(fdata.node_feature["n0"].reshape(*original_shape["n0" if "n0" in grid_keys else grid_keys[0]], *size))

    #         fig, ax1 = plt.subplots(figsize=(16,4),ncols=4)
    #         ###Boundary Location###
    #         mappable0 = ax1[0].imshow(a[:,:,0,0], cmap='viridis',
    #                                  aspect='auto',
    #                                  origin='lower')
    #         ax1[0].set_title('Boundary Map')
    #         ###Heatmap of Optimized Gas Location###
    #         mappable1 = ax1[1].imshow(a[:,:,0,1], cmap='viridis',
    #                                  interpolation="bicubic",
    #                                  aspect='auto',
    #                                  origin='lower')
    #         ax1[1].set_title('Initial Location')
    #         mappable2 = ax1[2].imshow(a[:,:,0,2], cmap='viridis',
    #                                  interpolation="bicubic",
    #                                  aspect='auto',
    #                                  origin='lower')
    #         ax1[2].set_title('X-Veloctiy')
    #         mappable3 = ax1[3].imshow(a[:,:,0,3], cmap='viridis',
    #                                  interpolation="bicubic",
    #                                  aspect='auto',
    #                                  origin='lower')
    #         ax1[3].set_title('Y-Veloctiy')
    #         fig.colorbar(mappable0, ax=ax1[0])
    #         fig.colorbar(mappable1, ax=ax1[1])
    #         fig.colorbar(mappable2, ax=ax1[2])
    #         fig.colorbar(mappable3, ax=ax1[3])
    #         fig.tight_layout()
    #         plt.show()

        t2 = time.time()
        preds, info = model(fdata, 
                            pred_steps=np.arange(target_length-preds_length, target_length),
                            #pred_steps=np.arange(1, n_rollout_steps+1), 
                            use_grads=args.use_grads, 
                            is_recons=is_recons, 
                            is_y_diff=args.is_y_diff, 
                            is_rollout=False,
                            static_data=stdata
                           )  # [n_nodes, pred_steps: n_rollout_steps, dyn_dims]

        preds, info = rollout(model, preds, info, fdata, is_recons)
        t3 = time.time()

        np_pred_target = to_np_array(preds["n0"].reshape(*original_shape["n0" if "n0" in grid_keys else grid_keys[0]], *preds["n0"].shape[1:]))
        if it == drawstep:
            pdf = PdfPages('./results/figures/predicted_trajectory.pdf')
            for j in range(preds_length):
                fig, ax = plt.subplots(figsize=(4,4))
                ###Heatmap of prediction of gas location###
                mappable0 = ax.imshow(np_pred_target[:,:,j,0], cmap='viridis',
                                         aspect='auto',
                                         origin='lower')
                ax.set_title('Predicted Target')
                fig.colorbar(mappable0, ax=ax)
                fig.tight_layout()
                pdf.savefig()
                plt.show()
            pdf.close()

        #calculate fractions and loss associated with the fractions
        pred_gas_fractions = outlet_loss(preds, masks, preds_length)    
        output = loss(pre_defined_fraction, torch.div(pred_gas_fractions, pred_gas_fractions.sum()))
        print("loss: ", output)

        if is_gpu:
            gpu_loss = output.cpu().detach().numpy()
            gpu_fraction1 = pred_gas_fractions[0].cpu().detach().numpy()
            gpu_fraction2 = pred_gas_fractions[1].cpu().detach().numpy()
            if gpu_loss < best_loss:
                best_loss = gpu_loss
                best_preds = preds
                best_fractions = [gpu_fraction1, gpu_fraction2]
                best_params = (lx1, rx1, rx2)

            print(pred_gas_fractions[0].cpu().detach().numpy())
            pred_fraction1.append(gpu_fraction1)
            target_fractions1.append(pre_defined_fraction[0].cpu().detach().numpy())

            pred_fraction2.append(gpu_fraction2)
            target_fractions2.append(pre_defined_fraction[1].cpu().detach().numpy())

            loss_list.append(gpu_loss)

        else:
            loss_list.append(output.detach().numpy())

        if it%time_draw==0:
            # draw output
            fig, ax = plt.subplots(figsize=(24,8),ncols=3)
            x = list(range(len(loss_list)))
            y = loss_list
            ax[0].plot(x, y)
            ax[0].set_yscale("log", base=10)
            ax[0].set_title("Loss b.w. Ground Truth and Prediced Targets")
            #plt.show()

            x_1 = list(range(len(pred_fraction1)))
            y_1 = pred_fraction1
            yt_1 = target_fractions1
            ax[1].plot(x_1, y_1)
            ax[1].plot(x_1, yt_1, '--')
            ax[1].set_yscale("log", base=10)
            ax[1].set_title("Loss b.w. Target and Prediced fractions at OUTLET 1")
            #plt.show()

            x_2 = list(range(len(pred_fraction2)))
            y_2 = pred_fraction2
            yt_2 = target_fractions2
            ax[2].plot(x_2, y_2)
            ax[2].plot(x_2, yt_2, '--')
            ax[2].set_yscale("log", base=10)
            ax[2].set_title("Loss b.w. Target and Prediced fractions at OUTLET 2")
            plt.show()

        optimizer.zero_grad()
        
        t0 = time.time()
        
        output.backward()
        optimizer.step()
        
        t1 = time.time()
        times.append([t1-t0, t3-t2])
        # plt.plot(times)
        # plt.show()

        print("current parameters and their gradients")
        print(lx1, lx1.grad)
        print(rx1, rx1.grad)
        print(rx2, rx2.grad)


    x = list(range(len(loss_list)))
    y = loss_list

    # Draw Transition of Loss
    plt.plot(x, y)
    plt.yscale("log", base=10)
    plt.title("Loss b.w. Ground Truth and Predicted Targets")
    plt.show()
    
    params_dict[str(i)] = {
        "initial_params": sample_space[sample_indices[i]],
        "pred_fractions": [pred_fraction1, pred_fraction2],
        "params_test": params_test,
        "loss_list": loss_list,
        "times": times
    }
    all_pred_fractions.append([pred_fraction1, pred_fraction2])
    all_test_params.append(params_test)
    all_losses.append(loss_list)
    all_times.append(times)
    
    
with open('latent_data.pkl', 'wb') as f:
    pickle.dump(params_dict, f)

In [ ]:
print(len(all_pred_fractions), len(all_pred_fractions[0][0]))

average_preds1 = []
average_preds2 = []

for j in range(iterations):
    for k in range(2):
        temp = []
        for i in range(len(all_pred_fractions)): 
            temp.append(all_pred_fractions[i][k][j])
        if k == 0:
            average_preds1.append(np.mean(np.array(temp)))
        if k == 1:
            average_preds2.append(np.mean(np.array(temp)))

plt.plot(average_preds1)
plt.ylim(0, 1)
plt.show()
plt.plot(average_preds2)
plt.ylim(0, 1)
plt.show()

In [ ]:
all_losses

average_losses = []
#average_times2 = []

for j in range(iterations):
    temp = []
    for i in range(len(all_pred_fractions)): 
        temp.append(all_losses[i][j])
    average_losses.append(np.mean(np.array(temp)))
    
plt.plot(average_losses)
plt.show()

In [ ]:
all_times

average_times = []
#average_times2 = []

for j in range(iterations):
    temp = []
    for i in range(len(all_times)): 
        temp.append(sum(all_times[i][j]))
    average_times.append(np.mean(np.array(temp)))
    
plt.plot(average_times)
plt.show()


In [ ]:
print(len(all_test_params), len(all_test_params[0]), len(all_test_params[0][0]))

In [ ]:
def ground_truth_test(params, beta):

    ims = []
    #list_final_fractions = []
    scenecount=1

    loop=75
    test_is_binary_thick_bound = True

    #pdf = PdfPages('./dataset/figures/optimized_trajectory.pdf')

    print(params)
    lx1, rx1, rx2, yrand, xrand = params
    rand = (yrand, xrand)
    for scene_index in range(scenecount):
        print("SCENE"+str(scene_index))
        thick_bound = make_thick_bound(lx1, rx1, rx2, beta=beta)
        if test_is_binary_thick_bound:
            binary_thick_bound = torch.where(thick_bound>0.5, 1., 0.)
        else:
            binary_thick_bound = thick_bound
        np_bound = to_np_array(binary_thick_bound)

        sim = FluidSimulation([128]*2, DomainBoundary([(True, True), (True, True)]), force_use_masks=True)
        sim._active_mask[0,:,:,0] = np_bound
        sim._fluid_mask[0,:,:,0] = np_bound
        sim._velocity_mask = sim._boundary.create_velocity_mask(sim._fluid_mask, sim._dimensions, sim._mac)

        res_sim = sim._fluid_mask.reshape((128,128))
        boundaries = np.argwhere(res_sim==0)
        ver_bound = boundaries[:,0]
        hor_bound = boundaries[:,1]

        x,y = np.meshgrid(np.linspace(0,128,129),np.linspace(0, 128, 129))

        print(sim)
        #plot_initial_velocity(sim)
        init_op_velocity, optimizable_velocity = initialize_velocity()
        #plot_init_op_velocity(init_op_velocity)
        divergent_velocity = apply_mask(sim, optimizable_velocity)
        #plot_velocity_with_mask(divergent_velocity)
        velocity = sim.divergence_free(divergent_velocity, solver=SparseCGPressureSolver(), accuracy=1e-5)
        velocity = sim.with_boundary_conditions(velocity)
        #plot_velocity_boundary_effect(velocity)
        #plot_vector_field(velocity)
        org_gas = orgdata.node_feature["n0"][:,:,1]
        rand_gas = randomize_gas(org_gas, rand)
        # init_gas = rand_gas.reshape([16384,1,1])
        init_gas = to_np_array(rand_gas.reshape([128, 128, 1]))
        # fig = plt.figure()
        # plt.imshow(init_gas.reshape([128,128]), origin='lower')
        # plt.show()
        array = init_gas
        init_op_density = StaggeredGrid(array)
        init_op_density = init_op_density.staggered.reshape((1,)+init_op_density.staggered.shape)

        #Advect
        advected_density = velocity.advect(init_op_density)
        #Visualize One-step Advected Gas
        fig = plt.figure()
        plt.imshow(advected_density[0,:,:,0], origin='lower')
        plt.show()

        loop_advected_density = advected_density
        loop_velocity = velocity

        vel_array = np.empty([129, 129, 2], np.float32)
        vel_array[...,0] = loop_velocity.staggered[0,:,:,0]
        vel_array[...,1] = loop_velocity.staggered[0,:,:,1]
        masks = outlet_masks(lx1, rx1, rx2)
        masks = (to_np_array(masks[0]), to_np_array(masks[1]))
        evaluate_densities = []
        evaluate_timestamps = [i for i in range(loop-preds_length, loop)]
        for frame in range(1, loop):
            loop_advected_density = loop_velocity.advect(loop_advected_density)
            loop_velocity = sim.divergence_free(loop_velocity, solver=SparseCGPressureSolver(), accuracy=1e-5)
            loop_velocity = sim.with_boundary_conditions(loop_velocity)
            vel_array = np.empty([129, 129, 2], np.float32)
            vel_array[...,0] = loop_velocity.staggered[0,:,:,0]
            vel_array[...,1] = loop_velocity.staggered[0,:,:,1]
            if frame in evaluate_timestamps:
                evaluate_densities.append(loop_advected_density[0,:,:,0].reshape(-1))
            # if True:
            #     fig, ax = plt.subplots()
            #     ax.imshow(loop_advected_density[0,:,:,0], origin='lower')
            #     ax.scatter(hor_bound, ver_bound, s=1, color="grey", marker=",")
            #     #pdf.savefig()
            #     plt.show()
        #pdf.close()    

        inners1 = []
        inners2 = []
        for i in range(preds_length):
            inners1.append(np.inner(to_np_array(evaluate_densities[i]), masks[0].reshape(-1)))
            inners2.append(np.inner(to_np_array(evaluate_densities[i]), masks[1].reshape(-1)))
        sim_gas_out1 = np.stack(inners1).sum()
        sim_gas_out2 = np.stack(inners2).sum()

        sim_pred_fractions = np.concatenate((sim_gas_out1.reshape((1,)), sim_gas_out2.reshape((1,))))
        final_fractions = np.divide(sim_pred_fractions, sim_pred_fractions.sum())
        print("final ratio over outlets: ", final_fractions)
        #list_final_fractions.append(final_fractions)
    return final_fractions

In [ ]:
index_beta = []
for i in range(len(params_test)):
    if i == 0:
        index_beta.append(beta_list[0])
    else: 
        index_beta.append(beta_list[test_time*i - 1])

all_fractions_test = []
for params_test in all_test_params:
    index = all_test_params.index(params_test)
    print(index)
    rand_direc = (sample_space[sample_indices[index]][3], sample_space[sample_indices[index]][4])
    fractions_test = [ground_truth_test(params_test[j]+rand_direc , index_beta[j]) for j in range(len(params_test))]
    all_fractions_test.append(fractions_test)

test_fractions_dict = {"all_fractions_test": all_fractions_test}
with open('latent_test_results.pkl', 'wb') as f:
    pickle.dump(test_fractions_dict, f)

In [ ]:
print(len(all_fractions_test), len(all_fractions_test[0]), len(all_fractions_test[0][0]))
print(test_time+1)
#all_pred_fractions

average_testpreds1 = []
average_testpreds2 = []

for j in range(int(iterations/test_time)+1):
    for k in range(2):
        temp = []
        for i in range(len(all_fractions_test)): 
            temp.append(all_fractions_test[i][j][k])
        if k == 0:
            average_testpreds1.append(np.mean(np.array(temp)))
        if k == 1:
            average_testpreds2.append(np.mean(np.array(temp)))

plt.plot(average_testpreds1)
plt.show()
plt.plot(average_testpreds2)
plt.show()

average_fractions_test = [average_testpreds1, average_testpreds2]

In [ ]:
deviations1 = []
deviations2 = []
for j in range(iterations):
    for k in range(2):
        temp = []
        for i in range(len(all_pred_fractions)): 
            temp.append(all_pred_fractions[i][k][j])
        if k == 0:
            deviations1.append(np.std(temp))
        elif k == 1:
            deviations2.append(np.std(temp))
            
len(deviations1)
    

In [ ]:
xs = np.arange(iterations)

fractions_lower = []

for i in range(iterations):
    if i == 0:
        fractions_lower.append(average_fractions_test[0][i])
    elif i%test_time == test_time - 1:
        print((i+1)/20)
        fractions_lower.append(average_fractions_test[0][int((i+1)/20)])
    else:
        fractions_lower.append(None)
        
series1 = np.array(fractions_lower).astype(np.double)
s1mask = np.isfinite(series1)

test_deviations1 = []
for j in range(int(iterations/test_time)+1):
    for k in range(2):
        temp = []
        for i in range(len(all_fractions_test)): 
            temp.append(all_fractions_test[i][j][k])
        if k == 0:
            test_deviations1.append(np.std(temp))
        #if k == 1:
        #    average_testpreds2.append(np.mean(np.array(temp)))      
len(test_deviations1)

error_deviations = []
for i in range(iterations):
    if i == 0:
        error_deviations.append(test_deviations1[i])
    elif i%test_time == test_time - 1:
        print((i+1)/20)
        error_deviations.append(test_deviations1[int((i+1)/20)])
    else:
        error_deviations.append(0.)


fig,ax = plt.subplots()
plt.ylim(-0.1,1.)
ax.plot(xs[s1mask], series1[s1mask], linestyle='-', marker='o', color="orange", label="Ground Truth")
ax.errorbar(xs[s1mask], series1[s1mask], yerr = test_deviations1, color="orange")
ax.plot(xs, average_preds1,color="blue", label="Predicted values") 
ax.errorbar(xs, average_preds1, yerr=deviations1, color="blue")
#ax.set_yticks([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7])
plt.xlabel("iteration")
plt.ylabel("fraction")
plt.legend()
plt.show()

In [ ]:
###Multiple Advections of Gas###
ims = []

scenecount=1

loop=75
test_is_binary_thick_bound = True

#pdf = PdfPages('./dataset/figures/optimized_trajectory.pdf')

print(best_params)
lx1, rx1, rx2 = best_params
for scene_index in range(scenecount):
    print("SCENE"+str(scene_index))
    thick_bound = make_thick_bound(lx1, rx1, rx2, beta=0.051)
    if test_is_binary_thick_bound:
        binary_thick_bound = torch.where(thick_bound>0.5, 1., 0.)
    else:
        binary_thick_bound = thick_bound
    np_bound = to_np_array(binary_thick_bound)

    sim = FluidSimulation([128]*2, DomainBoundary([(True, True), (True, True)]), force_use_masks=True)
    sim._active_mask[0,:,:,0] = np_bound
    sim._fluid_mask[0,:,:,0] = np_bound
    sim._velocity_mask = sim._boundary.create_velocity_mask(sim._fluid_mask, sim._dimensions, sim._mac)

    res_sim = sim._fluid_mask.reshape((128,128))
    boundaries = np.argwhere(res_sim==0)
    ver_bound = boundaries[:,0]
    hor_bound = boundaries[:,1]

    x,y = np.meshgrid(np.linspace(0,128,129),np.linspace(0, 128, 129))

    print(sim)
    plot_initial_velocity(sim)
    init_op_velocity, optimizable_velocity = initialize_velocity()
    plot_init_op_velocity(init_op_velocity)
    divergent_velocity = apply_mask(sim, optimizable_velocity)
    plot_velocity_with_mask(divergent_velocity)
    velocity = sim.divergence_free(divergent_velocity, solver=SparseCGPressureSolver(), accuracy=1e-5)
    velocity = sim.with_boundary_conditions(velocity)
    plot_velocity_boundary_effect(velocity)
    plot_vector_field(velocity)
    init_gas = to_np_array(orgdata.node_feature["n0"][:,:,1].reshape([128, 128, 1]))
    fig = plt.figure()
    plt.imshow(init_gas.reshape([128,128]), origin='lower')
    plt.show()
    array = init_gas
    init_op_density = StaggeredGrid(array)
    init_op_density = init_op_density.staggered.reshape((1,)+init_op_density.staggered.shape)

    #Advect
    advected_density = velocity.advect(init_op_density)
    #Visualize One-step Advected Gas
    fig = plt.figure()
    plt.imshow(advected_density[0,:,:,0], origin='lower')
    plt.show()

    loop_advected_density = advected_density
    loop_velocity = velocity

    vel_array = np.empty([129, 129, 2], np.float32)
    vel_array[...,0] = loop_velocity.staggered[0,:,:,0]
    vel_array[...,1] = loop_velocity.staggered[0,:,:,1]
    masks = outlet_masks(lx1, rx1, rx2)
    masks = (to_np_array(masks[0]), to_np_array(masks[1]))
    evaluate_densities = []
    evaluate_timestamps = [i for i in range(loop-preds_length, loop)]
    for frame in range(1, loop):
        loop_advected_density = loop_velocity.advect(loop_advected_density)
        loop_velocity = sim.divergence_free(loop_velocity, solver=SparseCGPressureSolver(), accuracy=1e-5)
        loop_velocity = sim.with_boundary_conditions(loop_velocity)
        vel_array = np.empty([129, 129, 2], np.float32)
        vel_array[...,0] = loop_velocity.staggered[0,:,:,0]
        vel_array[...,1] = loop_velocity.staggered[0,:,:,1]
        if frame in evaluate_timestamps:
            evaluate_densities.append(loop_advected_density[0,:,:,0].reshape(-1))
        if True:
            fig, ax = plt.subplots()
            ax.imshow(loop_advected_density[0,:,:,0], origin='lower')
            ax.scatter(hor_bound, ver_bound, s=1, color="grey", marker=",")
            #pdf.savefig()
            plt.show()
    #pdf.close()    

    inners1 = []
    inners2 = []
    for i in range(preds_length):
        inners1.append(np.inner(to_np_array(evaluate_densities[i]), masks[0].reshape(-1)))
        inners2.append(np.inner(to_np_array(evaluate_densities[i]), masks[1].reshape(-1)))
    sim_gas_out1 = np.stack(inners1).sum()
    sim_gas_out2 = np.stack(inners2).sum()

    sim_pred_fractions = np.concatenate((sim_gas_out1.reshape((1,)), sim_gas_out2.reshape((1,))))
    final_fractions = np.divide(sim_pred_fractions, sim_pred_fractions.sum())
    print("final ratio over outlets: ", final_fractions)
        


In [ ]:
params_test

In [ ]:
###Multiple Advections of Gas###
ims = []

scenecount=1

loop=75
test_is_binary_thick_bound = True

#pdf = PdfPages('./dataset/figures/optimized_trajectory.pdf')

print(best_params)
lx1, rx1, rx2 = params_test[0]
for scene_index in range(scenecount):
    print("SCENE"+str(scene_index))
    thick_bound = make_thick_bound(lx1, rx1, rx2, beta=0.051)
    if test_is_binary_thick_bound:
        binary_thick_bound = torch.where(thick_bound>0.5, 1., 0.)
    else:
        binary_thick_bound = thick_bound
    np_bound = to_np_array(binary_thick_bound)

    sim = FluidSimulation([128]*2, DomainBoundary([(True, True), (True, True)]), force_use_masks=True)
    sim._active_mask[0,:,:,0] = np_bound
    sim._fluid_mask[0,:,:,0] = np_bound
    sim._velocity_mask = sim._boundary.create_velocity_mask(sim._fluid_mask, sim._dimensions, sim._mac)

    res_sim = sim._fluid_mask.reshape((128,128))
    boundaries = np.argwhere(res_sim==0)
    ver_bound = boundaries[:,0]
    hor_bound = boundaries[:,1]

    x,y = np.meshgrid(np.linspace(0,128,129),np.linspace(0, 128, 129))

    print(sim)
    plot_initial_velocity(sim)
    init_op_velocity, optimizable_velocity = initialize_velocity()
    plot_init_op_velocity(init_op_velocity)
    divergent_velocity = apply_mask(sim, optimizable_velocity)
    plot_velocity_with_mask(divergent_velocity)
    velocity = sim.divergence_free(divergent_velocity, solver=SparseCGPressureSolver(), accuracy=1e-5)
    velocity = sim.with_boundary_conditions(velocity)
    plot_velocity_boundary_effect(velocity)
    plot_vector_field(velocity)
    init_gas = to_np_array(orgdata.node_feature["n0"][:,:,1].reshape([128, 128, 1]))
    fig = plt.figure()
    plt.imshow(init_gas.reshape([128,128]), origin='lower')
    plt.show()
    array = init_gas
    init_op_density = StaggeredGrid(array)
    init_op_density = init_op_density.staggered.reshape((1,)+init_op_density.staggered.shape)

    #Advect
    advected_density = velocity.advect(init_op_density)
    #Visualize One-step Advected Gas
    fig = plt.figure()
    plt.imshow(advected_density[0,:,:,0], origin='lower')
    plt.show()

    loop_advected_density = advected_density
    loop_velocity = velocity

    vel_array = np.empty([129, 129, 2], np.float32)
    vel_array[...,0] = loop_velocity.staggered[0,:,:,0]
    vel_array[...,1] = loop_velocity.staggered[0,:,:,1]
    masks = outlet_masks(lx1, rx1, rx2)
    masks = (to_np_array(masks[0]), to_np_array(masks[1]))
    evaluate_densities = []
    evaluate_timestamps = [i for i in range(loop-preds_length, loop)]
    for frame in range(1, loop):
        loop_advected_density = loop_velocity.advect(loop_advected_density)
        loop_velocity = sim.divergence_free(loop_velocity, solver=SparseCGPressureSolver(), accuracy=1e-5)
        loop_velocity = sim.with_boundary_conditions(loop_velocity)
        vel_array = np.empty([129, 129, 2], np.float32)
        vel_array[...,0] = loop_velocity.staggered[0,:,:,0]
        vel_array[...,1] = loop_velocity.staggered[0,:,:,1]
        if frame in evaluate_timestamps:
            evaluate_densities.append(loop_advected_density[0,:,:,0].reshape(-1))
        if True:
            fig, ax = plt.subplots()
            ax.imshow(loop_advected_density[0,:,:,0], origin='lower')
            ax.scatter(hor_bound, ver_bound, s=1, color="grey", marker=",")
            #pdf.savefig()
            plt.show()
    #pdf.close()    

    inners1 = []
    inners2 = []
    for i in range(preds_length):
        inners1.append(np.inner(to_np_array(evaluate_densities[i]), masks[0].reshape(-1)))
        inners2.append(np.inner(to_np_array(evaluate_densities[i]), masks[1].reshape(-1)))
    sim_gas_out1 = np.stack(inners1).sum()
    sim_gas_out2 = np.stack(inners2).sum()

    sim_pred_fractions = np.concatenate((sim_gas_out1.reshape((1,)), sim_gas_out2.reshape((1,))))
    final_fractions = np.divide(sim_pred_fractions, sim_pred_fractions.sum())
    print("final ratio over outlets: ", final_fractions)

